In [ ]:
!pip install -r requirements.txt

# upload embeddings to Pinecone

In [ ]:
from rag import PineconeEmbedder

index_name = "rag"
pinecone = PineconeEmbedder(index_name=index_name)

In [ ]:
models = ["paraphrase-distilroberta-base-v1", "intfloat/e5-base"]
models = ["intfloat/e5-base"]

# create the chunks of the dataset
chunks = pinecone.chunk_document("data/output.json", chunk_size=300, chunk_overlap=20)

In [ ]:
for model_path in models:
    # upload the chunks to the index with different namespace for each model
    pinecone.load_model(model_path=model_path, device="cuda")
    pinecone.upsert_parallel(chunks, namespace=model_path)

In [ ]:
import gc
import torch

# delete the model from the GPU
del pinecone.model
gc.collect()
torch.cuda.empty_cache()

# Run evaluation

In [ ]:
!python3 -m rag.eval